## Start Here

In [10]:
import json
import os

import re
from nltk.corpus import words as nltk_words
from wordfreq import zipf_frequency, word_frequency
import math

import itertools as it
import more_itertools as mit

from collections import deque
from typing import Any,Hashable


import networkx as nx
from networkx import DiGraph

import time
from datetime import timedelta
os.chdir("/Users/BeckyMarcusMacbook/Thesis/manual_work/")

In [3]:
level_2_chars = [
    ("cl",'a'),
    ('lz','h'),
    ('lz','n'),
    ('la', 'n'),
    ('la', 'h'),
    ('l','t'),
    ('i','l'),
    ('c','e'),
    ('e','c'),
    ('t','l'),
    ('ttl','m'),
    ("nz", 'm'),
    (r'v\b','y'),
    (r'r\b','y')
]
MASTER_CHARS = [
    #no mess ups probably
    ("vv", "w"),
    ("sv",'w'),
    ("cl", "d"),

    ("lll", "m"),
    ('lil','m'),
    ("nl", "m"),
    ("ln", "m"),
    ('zn','m'),

    ("il", "h"),
    ("ll", "h"),


    ("ll", "n"),
    ("rl", "n"),
    ("il", "n"),
    ("tl", "n"),
    


     ("tl", "u"),
    ("il", "u"),
     ("ll", "u"),
    #(r"s\b",""), # ones like these will remove any case of a pattern
]
all_chars = MASTER_CHARS+level_2_chars
first_chars = set([x[0]for x in all_chars])
pattern = "|".join([fr"\b[A-Za-z]*{char}[A-Za-z]*\b" for char in first_chars])

In [89]:
unfixed_dir = "/Users/BeckyMarcusMacbook/Thesis/TextCleaning/clean_text/E2"
def unknown_enough(word)->bool:
    return zipf_frequency(word,'en')<math.e and len(word)<20
texts = []
slightly_fixed = []
text_words:set[str] = set()
short_words = []
for file in os.listdir(unfixed_dir):
    path = os.path.join(unfixed_dir,file)
    text = open(path).read()
    texts.append(text)
    slightly_fixed1 = re.sub("1",'l',text)
    slightly_fixed = re.sub(r"[^\w\s]?","",slightly_fixed1)
    short = [w.lower() for w in re.findall(pattern,slightly_fixed) if len(w)<=4]
    short_words.extend(short)
    word_list:list[str] = [w.lower() for w in re.findall(pattern,slightly_fixed) if len(w)>=4]
    text_words.update(word_list)



In [56]:
from collections import Counter
import math
import pandas as pd
ct = Counter(short_words)

df=pd.DataFrame(ct.items(),columns=['word','freq']).sort_values('freq').reset_index(drop=True)
df['zfreq']=df['word'].apply(lambda w: zipf_frequency(w,'en'))
df['len'] = df['word'].apply(len)
df[(df['len']==4)&(df['zfreq']<3)].sort_values('zfreq').tail(40)

,word,freq,zfreq,len
289,leen,1,2.06,4
163,seel,1,2.07,4
117,coul,1,2.08,4
376,thal,1,2.08,4
134,hase,1,2.08,4
338,smal,1,2.10,4
446,onee,1,2.10,4
477,loth,2,2.12,4
30,cand,1,2.13,4
858,olle,22,2.13,4


In [ ]:


full_word_list = nltk_words.words()
corrections:dict[str,str] = json.load(open('corrections.json'))
prelim_known:set[str] = set([ *corrections.values(),*full_word_list])
print(f"Known: {len(prelim_known)}")

prelim_unknown = text_words.difference(prelim_known)
print(f"Prelim unknown: {len(prelim_unknown)}")
import math
def unknown_enough(word)->bool:
    return zipf_frequency(word,'en')<(math.e if len(word)>4 else 3) and len(word)<20
unknown_words = set([word.lower() for word in prelim_unknown if unknown_enough(word)])
print("Filtered:",len(unknown_words))

Known: 235938
Prelim unknown: 6819
Filtered: 5314


## Initialize Logger

In [59]:
import logging
NOTICE_LEVEL = 25
logging.addLevelName(NOTICE_LEVEL, "NOTICE")

# Create a custom logger class
class MyLogger(logging.Logger):
    def notice(self, message, *args, **kwargs):
        if self.isEnabledFor(NOTICE_LEVEL):
            self._log(NOTICE_LEVEL, message, args, **kwargs)

# Set the custom logger as the default logger class
logging.setLoggerClass(MyLogger)

# Create the logger instance once
logger:MyLogger = logging.getLogger(__name__) #type:ignore
logger.setLevel(logging.INFO)
if logger.hasHandlers():
    logger.handlers.clear()
logger.propagate = False

# Stream handler for NOTICE and above (console)
stream_handler = logging.StreamHandler()
stream_handler.setLevel(NOTICE_LEVEL)
stream_format = logging.Formatter('%(levelname)s - %(message)s')
stream_handler.setFormatter(stream_format)

# File handler for INFO and above (temp file)
temp_log_file = "E2/last_run.log"
if not os.path.exists(temp_log_file):
    with open(temp_log_file,'w') as f:
        f.write('')
file_handler = logging.FileHandler(temp_log_file)
file_handler.setLevel(logging.INFO)
file_format = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(file_format)

# Add handlers to the logger
logger.addHandler(file_handler)
logger.addHandler(stream_handler)

In [61]:

main_log_file = "E2/graph_runs.log"
temp_log_file = "E2/last_run.log"
if not os.path.exists(main_log_file):
    with open(main_log_file,'w') as f:
        f.write("")
if not os.path.exists(temp_log_file):
    with open(temp_log_file,'w') as f:
        f.write("")
        
def log_reset(temp_log_file:str=temp_log_file,notes:str ="",overwrite:bool=True):
    if overwrite:
        print("Overwriting")
    with open(temp_log_file,'w' if overwrite else 'a') as f:
        f.write("")
    logger.notice("LOGGER RESET - %s",notes)



In [124]:
{k:v for k,v in results.items() if v=="edmund"}

{'edlllund': 'edmund', 'edmtllld': 'edmund'}

In [86]:
zipf_frequency('conn','en')

3.38

In [109]:
unknown_words =UNKNOWN_WORDS
chars:list[tuple[str,str]] = MASTER_CHARS
default_max_iter=3
note = input("Note for the run: ")

iter_report_level = NOTICE_LEVEL
char_report_level = logging.INFO
word_report_level = logging.DEBUG
try:
    max_iteration = int(input("Max Iteration: "))
except ValueError:
    logger.warning("Input unrecognized, defaulting to %d",default_max_iter)
    max_iteration = default_max_iter
overwrite = re.search(r"yY", input("Do you want to overwrite the log file rather than appending to it? [y/n] ")) is not None

log_reset(notes=note,overwrite=overwrite)
logging.info("Initiating known words in graph")

G=DiGraph()
results ={}
G.add_nodes_from(((word,
                  dict(root=word,
                       root_unknown_words=set([word]),
                       final=None,
                       last_change = 0,
                       level=0,
                       )
                    )
                    for word in unknown_words)
                )
# next_iter_pq:PriorityQueue[Item] = PriorityQueue()
# for word in unknown_words:
#     next_iter_pq.put(Item(0,word))
run_start = time.monotonic()
made_change=True
#results.clear()
current_iter=0
false_pos=['hong']
logger.info("Max iteration: %d",max_iteration)
start_unknown_ct = len(unknown_words)
next_iter_queue = deque(unknown_words)
next_char_queue = deque()
dead_leaves = set()
    #next_char_pq:PriorityQueue[Item]= PriorityQueue()
try:
    while current_iter<max_iteration:
        dead_leaves=0


                
        # Start iteration
        current_iter_queue = next_iter_queue+next_char_queue
        next_iter_pq=deque()
        it_start_time = time.monotonic()
        current_iter+=1
        logger.notice("Starting Iteration %s",current_iter)
        info = {
            'nodes':G.number_of_nodes(),
            'edges':G.number_of_edges(),
            'results':len(results),
            'time':time.monotonic()
        }
        n_nodes_start = G.number_of_nodes()
        n_edges_start = G.number_of_edges()
        results_start = len(results)
        logger.notice("Starting with %d in queue - %d nodes and %d edges",len(current_iter_queue),n_nodes_start,n_edges_start)
        next_char_queue = current_iter_queue
        for old,new in MASTER_CHARS:
            ## Start Char
            char_start = time.monotonic()
            current_char_queue=next_char_queue
            next_char_queue = deque()
            old_tup = tuple(old)
            logger.info("Replacing '%s' with '%s'",old,new)
            start_result = len(results)
            start_nodes = G.number_of_nodes()
            start_edges = G.number_of_edges()
            #print(f"In results: {len(results)} out of {len(unknown_words)}")
            time_start = time.time()
            while current_char_queue:
                ### Start Word
                node_word = current_char_queue.pop()
                node_solved=False
                data:dict = G.nodes[node_word]
                og = data['root']
                logger.debug("Popping '%s' with root node %s", node_word,og)
                
                root_words:set[str] = data['root_unknown_words']
                right_word = results.get(mit.first_true([og,*root_words],None,lambda x: results.get(x,False)))
                if right_word is not None:
                    logger.info("On node '%s' - One of %d root words has path to real world '%s', updating all",node_word,len(root_words),right_word)
                    results.update({**{word:right_word for word in root_words}})
                    data.update({'final':right_word})
                    continue
                if data['last_change']<current_iter-1:
                    #means it can't be chagned anymore
                    logging.info("%s can't be changed further")
                    dead_leaves += 1
                any_change, node_solved = iterate_word(G,current_iter,node_word,old,new,og,root_words,data,next_iter_queue)
                
                if node_solved:
                    logger.debug("Continuing to next node word")
                    continue
                if not any_change:
                    logger.debug("'%s' not found in '%s', putting back in queue",old,node_word)
                else:
                    logger.debug("No known word found for %s, putting back in queue",node_word)
                    data.update({'last_change':current_iter})

                next_char_queue.append(node_word)
                ### End word


            char_elapsed = timedelta(seconds = time.monotonic()-char_start)
            char_edge_diff= G.number_of_edges() - start_edges
            char_node_diff = G.number_of_nodes() - start_nodes
            logger.info("Iteration %d - '%s' to '%s' - Time: %s - Results Added: %d - Edges Added: %d - Nodes Added: %d",
                        current_iter,old,new,char_elapsed,len(results)-start_result,char_edge_diff,char_node_diff)
            ## End char
        #report_end_iter(iter_report_level,G,current_iter,it_start_time,n_nodes_start,n_edges_start)
        logger.notice("After iteration %d, %d results found",current_iter,len(results))
        # End Iteration
    if not made_change:
        print(f"No more changes made after iteration {current_iter}")
    

except KeyboardInterrupt:
    reason = input("Reason for interrupting")
    logger.notice("Ending run: %s",reason)
    raise
except Exception as e:
    logging.error("Error: %s",e)
    raise
finally:
    elapsed = timedelta(seconds=time.monotonic() - run_start)
    still_out_there = [w for w in unknown_words if w not in results]

    logger.notice("Ran %d iterations over %s to get %d results out of %d",max_iteration,elapsed,len(results),start_unknown_ct )

NOTICE - LOGGER RESET - Changed thresholds for short words
NOTICE - Starting Iteration 1
NOTICE - Starting with 5730 in queue - 5730 nodes and 0 edges
NOTICE - After iteration 1, 1836 results found
NOTICE - Starting Iteration 2
NOTICE - Starting with 20276 in queue - 17558 nodes and 12758 edges
NOTICE - After iteration 2, 2120 results found
NOTICE - Starting Iteration 3
NOTICE - Starting with 39428 in queue - 26781 nodes and 30597 edges
NOTICE - After iteration 3, 2158 results found
NOTICE - Starting Iteration 4
NOTICE - Starting with 60835 in queue - 32072 nodes and 46365 edges
NOTICE - After iteration 4, 2164 results found
NOTICE - Starting Iteration 5
NOTICE - Starting with 82440 in queue - 34318 nodes and 55486 edges
NOTICE - After iteration 5, 2164 results found
NOTICE - Ran 5 iterations over 0:01:26.859853 to get 2164 results out of 5730


In [129]:
json.dump(results,open("E2/results_first_run.json",'w'))

## New funcs

In [248]:
still_out_there

{'preeminently',
 'eonlulereial',
 'freelottl',
 'reslllti',
 'trallspoltatioll',
 'worldconquerors',
 'lgeell',
 'tllouollt',
 'alliluate',
 'disinterestedlless',
 'cliainetrlcally',
 'jlldte',
 'inaslnuell',
 'irrialtioll',
 'ullwllolesollle',
 'selfeviclent',
 'illaliellable',
 'lzollaclaries',
 'iolcreasillb',
 'stllat',
 'ilnpllgll',
 'olullify',
 'llolv',
 'loollllcls',
 'wellnigh',
 'srell',
 'trtltn',
 'doetrille',
 'liljerty',
 'ellorlllouslt',
 'persoolal',
 'sstllen',
 'thlolloll',
 'eeolaomie',
 'colnpetitioul',
 'trallsl',
 'nlents',
 'apprellelesioll',
 'glorificatioll',
 'terlus',
 'othicll',
 'strotlbest',
 'offlland',
 'folbicl',
 'aioderll',
 'wellarranged',
 'beelzl',
 'indtlstlial',
 'illllabiteci',
 'lneclieal',
 'socalled',
 'reilledy',
 'tiolls',
 'septelilber',
 'folopportllllities',
 'secoolcl',
 'wlliell',
 'recluceci',
 'otllersl',
 'praetieally',
 'llgleal',
 'peronaoletlt',
 'welltodo',
 'geotraplly',
 'tellis',
 'lligllel',
 'eomonullity',
 'vitllin',
 'ph

In [337]:
level_2_chars = [
    ('l','T'),
    ('l',"R"),
    ('l',"I"),
    ('t','L'),
    ('i','L'),
    ('c','E'),
    ('e','C'),
    ("ml","RM"),
    (r'v\b','Y'),
    (r'r\b','Y'),
    (r'z\b','Y'),
    ("ll","M"),
]
level_1_chars = [
    #no mess ups probably
    ("vv", "W"),
    ("sv",'W'),
    ('xv','w'),
    ("cl", "D"),

    ("lll", "M"),
    ('ttl','M'),
    ('ltl','M'),
    ("nl", "M"),
    ("ill","M"),
    ('tll',"M"),
    ("rn","M"),

    ("ln", "M"),
    ('zn','M'),
    ('nz',"M"),
    ('ul',"M"),
    ("tn","M"),
    ('in',"M"),

    ("cl",'A'),
    ('lz','H'),
    ('lz','N'),
    ('la', 'N'),
    ('la', 'H'),

    ("il", "H"),
    ("ll", "H"),
    ("tl","H"),


    ("ll", "N"),
    ("rl", "N"),
    ("il", "N"),
    ("tl", "n"),
    
     ("tl", "U"),
    ("il", "U"),
     ("ll", "U"),
    #(r"s\b",""), # ones like these will remove any case of a pattern
]

In [234]:
'macle' in corrections

False

In [239]:
unfixed_dir = "/Users/BeckyMarcusMacbook/Thesis/TextCleaning/clean_text/E2"
from typing import Callable
texts = []
import math

def get_unknown_words(chars:list[tuple[str,str]],prelim_known:set[str],filter_func:Callable[[str],bool]|None=None):
    
    def unknown_enough(word)->bool:
        return zipf_frequency(word,'en')<(math.e if len(word)>4 else 3) and len(word)<20
    filter_func =unknown_enough if filter_func is None else filter_func
    text_words:set[str] = set()
    char_set = set([char[0] for char in chars])
    pattern = "|".join([fr"\b[A-Za-z]*{char}[A-Za-z]*\b" for char in char_set])
    for file in os.listdir(unfixed_dir):
        path = os.path.join(unfixed_dir,file)
        text = open(path).read()
        texts.append(text)
        slightly_fixed1 = re.sub("1",'l',text)
        slightly_fixed = re.sub(r"[^\w\s]?","",slightly_fixed1)
        #short = [w.lower() for w in re.findall(pattern,slightly_fixed) if len(w)<=4]
        #short_words.extend(short)
        word_list:list[str] = [w.lower() for w in re.findall(pattern,slightly_fixed) if len(w)>=4]
        text_words.update(word_list)
    print(f"Known: {len(prelim_known)}")
    prelim_unknown = text_words.difference(prelim_known)
    print(f"Prelim unknown: {len(prelim_unknown)}")

    unknown_words = set([word.lower() for word in prelim_unknown if filter_func(word)])
    print("Filtered:",len(unknown_words))
    return unknown_words, text_words



In [333]:
e=math.e
results:dict[str,str] ={}
thresholds = {
    4:3,
    3:5,
    2:5,
}

from collections import deque
def apply_t(s,old,new,i)->str:
    if i==-1:
        return re.sub(old,new,s)
    return s[:i] + new + s[i + len(old):] 
def known_enough(word:str)->tuple[bool,float]:
    zfreq = zipf_frequency(word,'en')
    return zfreq>thresholds.get(len(word),e), zfreq

def iterate_word(G:DiGraph,
                 current_iter:int,
                 node_word:str,
                 match_word:str,
                 old:str,
                 new:str,
                 og:str,
                 root_words:set,
                 node_data:dict[str,Any],
                 next_iter_queue:deque[str],
                 results:dict[str,str],
                 )->tuple[bool,bool,bool]:
    
    node_data = G.nodes[node_word]
    match_word = node_data['match_word']
    og = node_data['root']
    root_words = node_data['root_unknown_words']
    
    i=None
    node_solved=False
    new_edge = False
    for i in mit.locate(match_word,pred = lambda *x: mit.iequals(x,old),window_size=len(old)) if new!="" else [-1]:
        new_match:str|None = apply_t(node_word,old,new,i)
        new_word = new_match.lower()
        if not new_word in G:
            known, zfreq = known_enough(new_word)
            if known:
                logger.info("SUCCESS - Likely found a new word: %s with frequency %.2f",new_word,zfreq)
                G.add_node(new_word,root_unknown_words=root_words,final=new_word,root=og,originally_known=False,freq=zfreq,last_change=current_iter,match_word=new_match)
                node_solved=True
                logger.debug("Updating results for %d roots")
                results.update({root:new_word for root in root_words})

            else:
                logger.debug("Adding new word %s to graph and next next queue",new_word)
                G.add_node(new_word,root_unknown_words=root_words,final=None,root=og,originally_known=False,freq=zfreq,last_change=current_iter,match_word=new_match)
                next_iter_queue.append(new_word)
        elif not G.has_edge(node_word,new_word):
            new_word_node = G.nodes[new_word]
            new_word_roots:set = new_word_node['root_unknown_words'] #update so they share roots, only the first time they have an edge, however
            root_words.update(new_word_roots)
            final = new_word_node['final']

            if final is not None:
                logger.info("SUCCESS - Created word %s from %s which connects to real word %s",new_word, node_word,final )
                node_data.update({'final':final})

                results.update({root:final for root in root_words})
                node_solved=True
            else:
                logger.debug("%s in graph but not a final word, updating root words for next iteration")
                new_word_roots.update(root_words)
        else:
            logger.info("%s already connected to %s, continuing")
        if not G.has_edge(node_word,new_word):
            G.add_edge(node_word,new_word,old=old,new=new,idx=i)
            new_edge=not G.has_edge(new_word,node_word)
        if node_solved:
            return True,True,True
    
    return i is not None,False, new_edge

In [300]:
[(x,2*x) for x in range(10)]

[(0, 0),
 (1, 2),
 (2, 4),
 (3, 6),
 (4, 8),
 (5, 10),
 (6, 12),
 (7, 14),
 (8, 16),
 (9, 18)]

In [318]:
G.edges

OutEdgeView([(9, 18), (10, 13), (11, 16), (13, 15), (14, 16), (16, 17), (8, 11)])

In [307]:
G.edges

OutEdgeView([(2, 4), (3, 6), (4, 8), (5, 10), (6, 12), (7, 14), (8, 16), (9, 18), (0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 2)])

In [321]:
G=DiGraph()
G.add_nodes_from([*range(2,20)])
G.add_edges_from(list([(x,2*x) for x in range(10)]))
G.add_edges_from([(0,x) for x in range(0,5)])

n=0

while n<10:
    print("---")
    print("Nodes:",G.number_of_nodes())
    to_remove=set()
    new_edges=set()
    n+=1
    for parent,children in nx.bfs_successors(G,0,depth_limit=n):
        print(parent,children)
        change=False
        for node in children:
            new=node+n
            if node%2==0:
                change=True
            new_edges.add((node,new))
            #print(f"{parent}-->{node}")
        if not change:
            print(f"uh oh for {parent}")
            to_remove.add(parent)
    G.add_edges_from(new_edges)
    G.remove_nodes_from(to_remove)

---
Nodes: 20
0 [1, 2, 3, 4]
---
Nodes: 20
0 [1, 2, 3, 4]
3 [6]
4 [8, 5]
---
Nodes: 20
0 [1, 2, 3, 4]
3 [6, 5]
4 [8]
6 [12]
5 [10, 7]
8 [16]
---
Nodes: 20
0 [1, 2, 3, 4]
2 [5]
uh oh for 2
3 [6]
4 [8, 7]
5 [10]
6 [12, 9]
8 [16, 11]
7 [14]
10 [13]
uh oh for 10
12 [15]
uh oh for 12
9 [18]
16 [19]
uh oh for 16
---
Nodes: 19
0 [1, 3, 4]
1 [5]
uh oh for 1
3 [6, 7]
4 [8]
5 [9]
uh oh for 5
7 [14, 11]
9 [18, 13]
11 [15]
uh oh for 11
18 [22]
13 [17]
uh oh for 13
15 [19]
uh oh for 15
---
Nodes: 19
0 [3, 4]
3 [6, 7, 8]
4 [9]
uh oh for 4
7 [14, 12]
9 [18]
14 [19]
uh oh for 14
18 [22, 23]
19 [24]
22 [27]
uh oh for 22
---
Nodes: 23
0 [3]
uh oh for 0
3 [6, 7, 8, 9]
6 [12]
7 [13]
uh oh for 7
9 [18, 15]
18 [23, 24]
23 [29]
uh oh for 23
24 [30]
---
Nodes: 25


NetworkXError: The node 0 is not in the digraph.

In [353]:
def iterate_word(G:DiGraph,
                 current_iter:int,
                 node_word:str,
                 match_word:str,
                 old:str,
                 new:str,
                 og:str,
                 root_words:set,
                 node_data:dict[str,Any],
                 next_iter_queue:deque[str],
                 results:dict[str,str],
                 )->tuple[bool,bool,bool]:
    
    node_data = G.nodes[node_word]
    
    og = node_data['root']
    root_words = node_data['root_unknown_words']
def iterate_root(G:DiGraph,root_word,node_data,old,new,all_nodes:set[str]):
    i=None
    node_solved=False
    new_edge = False
    match_word = node_data['match_word']
    root=root_word
    last_level=set()
    parent=root
    for parent,nodes in nx.bfs_successors(G,root):
        
        if node in last_level:
            last_level.discard(node)
            parent=node

            continue
        last_level.add(node)


    edges_to_remove = set()
    all_nodes.update(G.nodes) #will include all of the ones to add as well
    nodes_to_add=dict()
    edges_to_add=set()
    for level_root,successors in nx.bfs_successors(G,root_word,depth_limit=current_iter):
        
        for node_word in successors:

            node_data=G.nodes[node_word]
            any_change=False
            for i in mit.locate(match_word,pred = lambda *x: mit.iequals(x,old),window_size=len(old)) if new!="" else [-1]:
                new_match:str|None = apply_t(node_word,old,new,i)
                new_word = new_match.lower()
                if not new_word in all_nodes:
                    known, zfreq = known_enough(new_word)
                    node_to_add=dict(
                        match_word=new_match,
                        root=root,
                        parent=level_root,
                        freq=zfreq
                    )
                    if known:
                        logger.info("SUCCESS - Likely found a new word: %s with frequency %.2f",new_word,zfreq)
                        node_to_add.update({'last_change':-1})
                        #logger.debug("Updating results for %d roots")
                        results[root_word]=new_word
                        #maybe continue since we don't need the edge
                    else:
                        node_to_add.update({
                            "last_change":current_iter
                        }
                        )
                        
                        #next_iter_queue.append(new_word)
                    all_nodes.add(new_word)
                elif not G.has_edge(node_word,new_word):
                    new_word_node = G.nodes[new_word]
                    new_og = new_word_node['root']
                    if new_og!=og:
                        final = results.get(new_og)
                        if final is not None:
                            logger.info("SUCCESS - Created word %s from %s which connects to real word %s",new_word, node_word,final)
                            results[root_word]=final
                            node_solved=True
                else:
                    logger.debug("%s already connected to %s, continuing")
                if any_change:
                    node_data.update({'last_change':current_iter})
             
    
    return i is not None,False, new_edge

SyntaxError: expression expected after dictionary key and ':' (2705599467.py, line 66)

In [ ]:
G =DiGraph()
G.add_nodes_from([1,2,3,4,5])
G.add_edge(1,2)
G.add_edge(1,4)
G.add_edge(4,3)
G.add_edge(2,6)
G.add_edge(4,5)
root=1
last_level=set()
parent=root
for node in mit.collapse(nx.bfs_successors(G,root)):
    if node in last_level:
        last_level.discard(node)
        parent=node
        continue
    last_level.add(node)


RuntimeError: dictionary changed size during iteration

In [338]:
from collections import defaultdict

iter_report_level = NOTICE_LEVEL
char_report_level = logging.INFO
word_report_level = logging.DEBUG

def word_correcting(unknown_words:set[str],
                    chars:list[tuple[str,str]],
                    max_iteration:int,
                    start_G:None|DiGraph= None,
                    start_results:dict[str,str]|None=None,
                    )->tuple[dict[str,str],set[str],DiGraph]:
    logger.notice("Starting run")
    G=DiGraph() if start_G is None else start_G
    results:dict[str,str] = start_results.copy() if start_results is not None else {}
    for word in unknown_words:
        if word not in G:
            G.add_node(word,
                        root=word,
                        match_word=word,
                        root_unknown_words=set([word]),
                        final=None,
                        last_change = 0,
                        level=0,
                            )
    nx.set_node_attributes(G,0,'last_change')
    run_start = time.monotonic()
    made_change=True
    current_iter=0
    logger.info("Max iteration: %d",max_iteration)
    start_unknown_ct = len(unknown_words)
    next_iter_queue = deque(unknown_words)
    next_char_queue = deque()
    dead_leaves = set()
    last_char_change = defaultdict(int)
        #next_char_pq:PriorityQueue[Item]= PriorityQueue()
    try:
        while current_iter<max_iteration:
            dead_leaves=0
            # Start iteration
            current_iter_queue = next_iter_queue+next_char_queue
            next_iter_queue:deque[str]=deque()
            it_start_time = time.monotonic()
            current_iter+=1
            logger.notice("Starting Iteration %s",current_iter)
            # info = {
            #     'nodes':G.number_of_nodes(),
            #     'edges':G.number_of_edges(),
            #     'results':len(results),
            #     'time':time.monotonic()
            # }
            n_nodes_start = G.number_of_nodes()
            n_edges_start = G.number_of_edges()
            results_start = len(results)
            logger.notice("Starting with %d in queue - %d nodes and %d edges",len(current_iter_queue),n_nodes_start,n_edges_start)
            next_char_queue = current_iter_queue
            for char in chars:
                old,new=char
                ## Start Char
                char_start = time.monotonic()
                current_char_queue=next_char_queue
                next_char_queue = deque()
                logger.info("Replacing '%s' with '%s'",old,new)
                start_result = len(results)
                start_nodes = G.number_of_nodes()
                start_edges = G.number_of_edges()
                #print(f"In results: {len(results)} out of {len(unknown_words)}")
                if last_char_change[char]<current_iter-1:
                    logger.info("Skipping %s to %s",old,new)
                    continue
                any_any_edge=False
                while current_char_queue:
                    ### Start Word
                    node_word = current_char_queue.pop()
                    node_solved=False
                    data:dict = G.nodes[node_word]
                    og = data['root']
                    logger.debug("Popping '%s' with root node %s", node_word,og)
                    
                    root_words:set[str] = data['root_unknown_words']
                    right_word = results.get(mit.first_true(root_words,"",lambda x: results.get(x,False)))
                    if right_word is not None:
                        logger.info("On node '%s' - One of %d root words has path to real world '%s', updating all",node_word,len(root_words),right_word)
                        results.update({word:right_word for word in root_words})
                        data.update({'final':right_word})
                        continue
                    if data['last_change']<current_iter-1:
                        #means it can't be chagned anymore
                        logging.info("%s can't be changed further")
                        dead_leaves += 1
                    match_word = data['match_word']
                    any_change, node_solved, any_edge = iterate_word(G,current_iter,node_word,match_word,old,new,og,root_words,data,next_iter_queue,results)
                    if any_edge:
                        any_any_edge=True
                    if node_solved:
                        logger.debug("Continuing to next node word")
                        continue
                    if not any_change:
                        logger.debug("'%s' not found in '%s', putting back in queue",old,node_word)
                    else:
                        logger.debug("No known word found for %s, putting back in queue",node_word)
                        data.update({'last_change':current_iter})

                    next_char_queue.append(node_word)
                    ### End word

                if not any_any_edge:
                    logger.info("Transformation'%s' to '%s' did not make any new edges",old,new)
                else:
                    last_char_change[char]+=1
                char_elapsed = timedelta(seconds = time.monotonic()-char_start)
                char_edge_diff= G.number_of_edges() - start_edges
                char_node_diff = G.number_of_nodes() - start_nodes
                logger.info("Iteration %d - '%s' to '%s' - Time: %s - Results Added: %d - Edges Added: %d - Nodes Added: %d",
                            current_iter,old,new,char_elapsed,len(results)-start_result,char_edge_diff,char_node_diff)
                ## End char
            #report_end_iter(iter_report_level,G,current_iter,it_start_time,n_nodes_start,n_edges_start)
            
            char_elapsed = timedelta(seconds = time.monotonic()-it_start_time)
            char_edge_diff= G.number_of_edges() - n_edges_start
            char_node_diff = G.number_of_nodes() - n_nodes_start
            logger.log(iter_report_level,"End of Iteration %d:  - Time: %s - Results Added: %d - Edges Added: %d - Nodes Added: %d",
                        current_iter,char_elapsed,len(results)-results_start,char_edge_diff,char_node_diff)
            logger.notice("After iteration %d, %d results found",current_iter,len(results))
            # End Iteration
        if not made_change:
            print(f"No more changes made after iteration {current_iter}")
        

    except KeyboardInterrupt:
        reason = input("Reason for interrupting")
        logger.notice("Ending run: %s",reason)
        raise
    except Exception as e:
        
        logging.error("Error: %s,",e)
        raise
    else:
        elapsed = timedelta(seconds=time.monotonic() - run_start)
        still_out_there = set([w for w in unknown_words if w not in results])

        logger.notice("Ran %d iterations over %s to get %d results out of %d",max_iteration,elapsed,len(results),start_unknown_ct )
        return results,still_out_there,G

In [ ]:
log_reset(overwrite=True)

In [339]:
manual_corrections = json.load(open('corrections.json','r'))
prelim_known = set([*manual_corrections.values()])
level_1_unknown,all_text = get_unknown_words(level_1_chars,prelim_known)
level_1_results, still_out_there, first_G = word_correcting(level_1_unknown,level_1_chars,6,start_results = manual_corrections)

NOTICE - Starting run
NOTICE - Starting Iteration 1
NOTICE - Starting with 4233 in queue - 4233 nodes and 0 edges


Known: 283
Prelim unknown: 5731
Filtered: 4233


NOTICE - End of Iteration 1:  - Time: 0:00:01.043013 - Results Added: 1792 - Edges Added: 14827 - Nodes Added: 13994
NOTICE - After iteration 1, 2139 results found
NOTICE - Starting Iteration 2
NOTICE - Starting with 14949 in queue - 18227 nodes and 14827 edges
NOTICE - End of Iteration 2:  - Time: 0:00:03.553364 - Results Added: 295 - Edges Added: 24657 - Nodes Added: 12773
NOTICE - After iteration 2, 2434 results found
NOTICE - Starting Iteration 3
NOTICE - Starting with 22432 in queue - 31000 nodes and 39484 edges
NOTICE - End of Iteration 3:  - Time: 0:00:05.845902 - Results Added: 44 - Edges Added: 22894 - Nodes Added: 7511
NOTICE - After iteration 3, 2478 results found
NOTICE - Starting Iteration 4
NOTICE - Starting with 25233 in queue - 38511 nodes and 62378 edges
NOTICE - End of Iteration 4:  - Time: 0:00:07.770992 - Results Added: 2 - Edges Added: 15362 - Nodes Added: 3295
NOTICE - After iteration 4, 2480 results found
NOTICE - Starting Iteration 5
NOTICE - Starting with 27002

In [351]:
json.dump(pd.DataFrame(level_1_results.items()).set_index(0).sort_index().to_dict(),open("E2/jan28_results1.json",'w'))

In [349]:
df = 
df

{1: {'aacl': 'add',
  'abolisll': 'abolish',
  'absellce': 'absence',
  'absorbillg': 'absorbing',
  'accolllpanied': 'accompanied',
  'accollnt': 'account',
  'accollntillg': 'accounting',
  'accompallied': 'accompanied',
  'accompaniecl': 'accompanied',
  'accotlllt': 'account',
  'accumulatillg': 'accumulating',
  'achievemellts': 'achievements',
  'aclam': 'adam',
  'aclams': 'adams',
  'aclapt': 'adapt',
  'aclaptecl': 'adapted',
  'aclatll': 'adam',
  'aclcl': 'add',
  'aclded': 'added',
  'aclequate': 'adequate',
  'acljustilag': 'adjusting',
  'acljustlllellts': 'adjustments',
  'acllievement': 'achievement',
  'acllievenlent': 'achievement',
  'acllllirably': 'admirably',
  'aclvance': 'advance',
  'aclvantage': 'advantage',
  'aclvantages': 'advantages',
  'aclvocacy': 'advocacy',
  'actioll': 'action',
  'actiotl': 'action',
  'acttlal': 'actual',
  'adain': 'adam',
  'adaln': 'adam',
  'adalnailt': 'adamant',
  'adattls': 'adams',
  'adazn': 'adam',
  'adclressilzg': 'addre

In [348]:
df = pd.DataFrame(level_1_results.items()).set_index(0).sort_index().to_dict()
df

{1: {'aacl': 'add',
  'abolisll': 'abolish',
  'absellce': 'absence',
  'absorbillg': 'absorbing',
  'accolllpanied': 'accompanied',
  'accollnt': 'account',
  'accollntillg': 'accounting',
  'accompallied': 'accompanied',
  'accompaniecl': 'accompanied',
  'accotlllt': 'account',
  'accumulatillg': 'accumulating',
  'achievemellts': 'achievements',
  'aclam': 'adam',
  'aclams': 'adams',
  'aclapt': 'adapt',
  'aclaptecl': 'adapted',
  'aclatll': 'adam',
  'aclcl': 'add',
  'aclded': 'added',
  'aclequate': 'adequate',
  'acljustilag': 'adjusting',
  'acljustlllellts': 'adjustments',
  'acllievement': 'achievement',
  'acllievenlent': 'achievement',
  'acllllirably': 'admirably',
  'aclvance': 'advance',
  'aclvantage': 'advantage',
  'aclvantages': 'advantages',
  'aclvocacy': 'advocacy',
  'actioll': 'action',
  'actiotl': 'action',
  'acttlal': 'actual',
  'adain': 'adam',
  'adaln': 'adam',
  'adalnailt': 'adamant',
  'adattls': 'adams',
  'adazn': 'adam',
  'adclressilzg': 'addre

In [261]:
filtered_G_nodes = set([node for node,data in first_G.nodes.items() if not set(data['root_unknown_words']).isdisjoint(still_out_there)])

In [265]:
prelim_known = set(level_1_results.keys())
level_2_unknown,all_text = get_unknown_words(level_2_chars,prelim_known)
all_unknown=level_2_unknown.union(still_out_there,filtered_G_nodes)
level_2_results, still_still_out_there, G2 = word_correcting(all_unknown,level_2_chars,6,start_results = level_1_results,start_G=first_G)

Known: 2428
Prelim unknown: 8615
Filtered: 3333


NOTICE - Starting run
NOTICE - Starting Iteration 1
NOTICE - Starting with 27883 in queue - 44529 nodes and 82864 edges
NOTICE - End of Iteration 1:  - Time: 0:00:16.051799 - Results Added: 688 - Edges Added: 192530 - Nodes Added: 191840
NOTICE - After iteration 1, 3116 results found
NOTICE - Starting Iteration 2
NOTICE - Starting with 213343 in queue - 236369 nodes and 275394 edges
NOTICE - End of Iteration 2:  - Time: 0:00:34.985373 - Results Added: 76 - Edges Added: 1301278 - Nodes Added: 678185
NOTICE - After iteration 2, 3192 results found
NOTICE - Starting Iteration 3
NOTICE - Starting with 678138 in queue - 914554 nodes and 1576672 edges
NOTICE - Ending run: too big


KeyboardInterrupt: 

In [195]:
level_1_results['made']

KeyError: 'made'

In [191]:
len(first_first_results)

2164

In [227]:
'macle' in all_text


True

In [228]:
'macle' in level_1_unknown

False

In [217]:
'universites' in first_G
first_G.nodes['universities']

{'root_unknown_words': {'ulliversities'},
 'final': 'universities',
 'root': 'ulliversities',
 'originally_known': False,
 'freq': 4.36,
 'last_change': 1,
 'match_word': 'uNiversities'}

In [214]:
first_first_results = json.load(open("/Users/BeckyMarcusMacbook/Thesis/manual_work/E2/results_first_run.json"))
still_out_there.difference(first_first_results.keys())

{'fillanee',
 'colll',
 'llighel',
 'laeveltlleless',
 'preeminently',
 'eonlulereial',
 'freelottl',
 'aclnlinistlatioll',
 'llunallitr',
 'labol',
 'clltzllinbest',
 'trllek',
 'reslllti',
 'lenrellclers',
 'beeollle',
 'ldossibilities',
 'trallspoltatioll',
 'worldconquerors',
 'fortunebuilding',
 'quiekll',
 'lgeell',
 'tllouollt',
 'alliluate',
 'disinterestedlless',
 'llavirg',
 'cliainetrlcally',
 'pllllic',
 'amotlgllunlan',
 'jlldte',
 'inaslnuell',
 'llowerler',
 'tlwe',
 'irrialtioll',
 'eanllot',
 'ullwllolesollle',
 'selfeviclent',
 'illaliellable',
 'lzollaclaries',
 'llts',
 'htilltxs',
 'hulllalwity',
 'cleserwrilig',
 'xvitll',
 'elelaouilced',
 'consttlaires',
 'clireeting',
 'iolcreasillb',
 'stllat',
 'luoderll',
 'delllad',
 'ilnpllgll',
 'lollls',
 'proteetiotl',
 'conaproollise',
 'orgalliza',
 'olullify',
 'llolv',
 'xnorally',
 'llistoryr',
 'loollllcls',
 'wellnigh',
 'srell',
 'trtltn',
 'clisplacine',
 'doetrille',
 'liljerty',
 'vlpsrarcl',
 'llnclertalce',

In [170]:

level_2_results, still_still_out_there, G2 = word_correcting(still_out_there,level_2_chars,6,first_G)

NOTICE - Starting run
NOTICE - Starting Iteration 1
NOTICE - Starting with 3158 in queue - 34607 nodes and 59002 edges
NOTICE - Transformation'cl' to 'a' did not make any new edges
NOTICE - Transformation'lz' to 'h' did not make any new edges
NOTICE - Transformation'lz' to 'n' did not make any new edges
NOTICE - Transformation'la' to 'n' did not make any new edges
NOTICE - Transformation'la' to 'h' did not make any new edges
NOTICE - Transformation'l' to 't' did not make any new edges
NOTICE - Transformation't' to 'l' did not make any new edges
NOTICE - Transformation'c' to 'e' did not make any new edges
NOTICE - Transformation'ttl' to 'm' did not make any new edges
NOTICE - Transformation'nz' to 'm' did not make any new edges
NOTICE - Transformation'v\b' to 'y' did not make any new edges
NOTICE - Transformation'r\b' to 'y' did not make any new edges
NOTICE - End of Iteration 1:  - Time: 0:00:00.941479 - Results Added: 544 - Edges Added: 14012 - Nodes Added: 13622
NOTICE - After iterat

KeyboardInterrupt: 

Exception ignored in: 'zmq.backend.cython._zmq.Frame.__del__'
Traceback (most recent call last):
  File "_zmq.py", line 160, in zmq.backend.cython._zmq._check_rc
KeyboardInterrupt: 
NOTICE - Transformation'c' to 'e' did not make any new edges
NOTICE - Transformation'e' to 'c' did not make any new edges
NOTICE - Transformation'ttl' to 'm' did not make any new edges
NOTICE - Transformation'nz' to 'm' did not make any new edges
NOTICE - Transformation'v\b' to 'y' did not make any new edges
NOTICE - Transformation'r\b' to 'y' did not make any new edges
NOTICE - End of Iteration 2:  - Time: 0:00:08.637001 - Results Added: 70 - Edges Added: 71434 - Nodes Added: 32457
NOTICE - After iteration 2, 614 results found
NOTICE - Starting Iteration 3
NOTICE - Starting with 47092 in queue - 80686 nodes and 144448 edges
NOTICE - Transformation'cl' to 'a' did not make any new edges
NOTICE - Transformation'lz' to 'h' did not make any new edges
NOTICE - Transformation'lz' to 'n' did not make any new edges

KeyboardInterrupt: 

In [163]:
new_results = {k:v for k,v in level_2_results.items() if k not in level_1_results}
new_results

{'cclll': 'call',
 'idecll': 'ideal',
 'cclse': 'case',
 'hcld': 'had',
 'hclve': 'have',
 'natiolzs': 'nations',
 'prilzciple': 'principle',
 'lzecessary': 'necessary',
 'filzd': 'find',
 'existilzg': 'existing',
 'ilzterests': 'interests',
 'alzd': 'and',
 'alzcl': 'and',
 'tlze': 'the',
 'olze': 'one',
 'alolzg': 'along',
 'tlzat': 'that',
 'lzlally': 'many',
 'gelzeral': 'general',
 'tlzeir': 'their',
 'plzblic': 'public',
 'wlzose': 'whose',
 'wrolzg': 'wrong',
 'nomilzally': 'nominally',
 'investigatilzg': 'investigating',
 'gelzerally': 'generally',
 'learnilag': 'learning',
 'opiniolas': 'opinions',
 'represelat': 'represent',
 'slaows': 'snows',
 'laeeded': 'needed',
 'laeither': 'neither',
 'uladue': 'undue',
 'alacl': 'and',
 'alad': 'and',
 'fitlaess': 'fitness',
 'judgmelat': 'judgment',
 'growilag': 'growing',
 'lnealas': 'means',
 'alolae': 'alone',
 'collapetitioll': 'competition',
 'celatllry': 'century',
 'interestilag': 'interesting',
 'illterestilag': 'interesting',

In [159]:
len(still_still_out_there)

2516

In [94]:
results_full = json.load(open("E2/results.json"))

In [95]:
not_in_results = set()

In [110]:
r1=set(results_full.keys())
r2=set(results.keys())
in_json = r1.difference(r2)
not_json=r2.difference(r1)
in_json ={key:results_full[key] for key in in_json}
not_json = {key:results[key] for key in not_json}

In [116]:
zipf_frequency('no','en')

6.35

In [123]:
json_not_corrections = {key:in_json[key] for key in set(in_json)-set(corrections) if key not in UNKNOWN_WORDS}
json_not_corrections

{'lillers': 'liners',
 'eatlles': 'eames',
 'llolclers': 'holders',
 'llolll': 'nom',
 'witllessed': 'witnessed',
 'epocll': 'epoch',
 'ellgaged': 'engaged',
 'fashionecl': 'fashioned',
 'increasillg': 'increasing',
 'inclividualized': 'individualized',
 'plallned': 'planned',
 'slrlitll': 'smith',
 'illfected': 'infected',
 'lleld': 'held',
 'salld': 'sand',
 'suillg': 'suing',
 'pllllldered': 'plundered',
 'ralll': 'ram',
 'collllected': 'connected',
 'sllrprised': 'surprised',
 'lnaster': 'master',
 'combilled': 'combined',
 'wiclelled': 'widened',
 'llollg': 'hong',
 'grall': 'gran',
 'cleluded': 'deluded',
 'fralne': 'frame',
 'greell': 'green',
 'sioll': 'sion',
 'lneal': 'meal',
 'aclapted': 'adapted',
 'cecled': 'ceded',
 'billd': 'bind',
 'llationality': 'nationality',
 'silllple': 'simple',
 'ullderstalld': 'understand',
 'tilnid': 'timid',
 'reclllced': 'reduced',
 'livelillood': 'livelihood',
 'colltrolled': 'controlled',
 'climinished': 'diminished',
 'lelll': 'lem',
 'bey

In [111]:
not_json

{'reputatioll': 'reputation',
 'wellbeillg': 'wellbeing',
 'llence': 'hence',
 'tsvo': 'two',
 'elilpires': 'empires',
 'svorse': 'worse',
 'lntlst': 'must',
 'cllallgecl': 'changed',
 'tellt': 'tent',
 'meails': 'means',
 'macle': 'made',
 'isslle': 'issue',
 'lltlnlall': 'human',
 'atll': 'am',
 'becolne': 'become',
 'clriven': 'driven',
 'oilt': 'out',
 'preselltecl': 'presented',
 'svllile': 'while',
 'increasecl': 'increased',
 'trainiilg': 'training',
 'adazn': 'adam',
 'svith': 'with',
 'fllll': 'full',
 'allilllal': 'animal',
 'coilsequence': 'consequence',
 'lnasterpiece': 'masterpiece',
 'llumber': 'humber',
 'nlay': 'may',
 'positioil': 'position',
 'inust': 'must',
 'becl': 'bed',
 'mtlst': 'must',
 'llpbrillgillg': 'upbringing',
 'adlnission': 'admission',
 'determilles': 'determines',
 'rutllless': 'ruthless',
 'reqllired': 'required',
 'beiilg': 'being',
 'tlot': 'not',
 'moiley': 'money',
 'osvn': 'own',
 'turil': 'turn',
 'coilllllollplace': 'commonplace',
 'iluman': '

In [114]:
in_json

{'lillers': 'liners',
 'cclll': 'ccm',
 'eatlles': 'eames',
 'lllal': 'mal',
 'llolclers': 'holders',
 'colll': 'com',
 'llolll': 'nom',
 'eollege': 'college',
 'witllessed': 'witnessed',
 'epocll': 'epoch',
 'ellgaged': 'engaged',
 'lle': 'he',
 'ltlon': 'mon',
 'fashionecl': 'fashioned',
 'increasillg': 'increasing',
 'inclividualized': 'individualized',
 'plallned': 'planned',
 'slrlitll': 'smith',
 'illfected': 'infected',
 'lleld': 'held',
 'llo': 'no',
 'salld': 'sand',
 'suillg': 'suing',
 'colulnercial': 'commercial',
 'nlull': 'mull',
 'pllllldered': 'plundered',
 'ralll': 'ram',
 'collllected': 'connected',
 'sllrprised': 'surprised',
 'lnaster': 'master',
 'combilled': 'combined',
 'wiclelled': 'widened',
 'llollg': 'hong',
 'grall': 'gran',
 'fralne': 'frame',
 'cleluded': 'deluded',
 'lilce': 'luce',
 'greell': 'green',
 'sioll': 'sion',
 'lneal': 'meal',
 'aclapted': 'adapted',
 'cecled': 'ceded',
 'otl': 'on',
 'lllle': 'hue',
 'tuall': 'mall',
 'colllluercial': 'commerc

In [79]:
results_full = json.load(open("E2/results.json"))
difference = set(results_full.keys()).difference(results)
print(f"Num changed: {len(difference)}")
# results_full.update(results)
# len(results_full)

Num changed: 0


In [76]:
with open("E2/results.json",'w') as f:
    json.dump({**results,**corrections},f)

In [352]:
results_full = json.load(open("E2/CHECKED.json"))

def match_case(original, replacement):
    """
    Adjust the replacement text to match the case of the original text.
    """
    if original.isupper():
        return replacement.upper()
    elif original.istitle():
        return replacement.title()
    elif original.islower():
        return replacement.lower()
    else:
        return replacement

def replace_with_case_sensitivity(text, replacements):
    """
    Replace occurrences of lowercase keys in the text with their respective values, maintaining case.
    """
    # Build regex pattern with all lowercase keys
    pattern = re.compile(r'\b(' + '|'.join(re.escape(k) for k in replacements.keys()) + r')\b', re.IGNORECASE)

    def replacement_function(match):
        matched_text = match.group(0)  # Get the actual matched text from input
        lower_matched = matched_text.lower()  # Convert to lowercase to match dictionary keys
        replacement_value = replacements.get(lower_matched, matched_text)  # Get replacement or keep original
        return match_case(matched_text, replacement_value)
    return pattern.sub(replacement_function, text)

folder = '/Users/BeckyMarcusMacbook/Thesis/TextCleaning/clean_text/E2'
pre_fix = '/Users/BeckyMarcusMacbook/Thesis/manual_work/E2/problematic_unfixed'
if not os.path.exists(pre_fix):
    os.makedirs(pre_fix)
new_folder = '/Users/BeckyMarcusMacbook/Thesis/manual_work/E2/problematic_fixed'
if not os.path.exists(new_folder):
    os.makedirs(new_folder)
all_words = []
for file in sorted(os.listdir(folder)):
    if file[0] == '.':
        continue
    disc,year,num,pagetxt = file.split("-")
    year = int(year)
    if year<1904 and year>=1900:
        path = os.path.join(folder, file)
        text=  open(path).read()
        #os.symlink(path,os.path.join(pre_fix,file))
        new_text = replace_with_case_sensitivity(text,results_full)
        if text!=new_text:
            with open(os.path.join(new_folder,file),'w') as f:
                f.write(new_text)


In [331]:
"illaterial" in results_full

False

# Break

In [55]:
word = "hellocl ancly meowcl"
re.sub(r"\b(\w+)(cl)\b",lambda match:match.group(1)+"d",word)

'hellod ancly meowd'

In [58]:
corrections

{'betweell': 'between',
 'ecollomics': 'economics',
 'influellce': 'influence',
 'opillion': 'opinion',
 'mealls': 'means',
 'presellt': 'present',
 'ill': 'in',
 'amollg': 'among',
 'tlle': 'the',
 'comlllercial': 'commercial',
 'conditiolls': 'conditions',
 'oll': 'on',
 'witll': 'with',
 'olles': 'ones',
 'econolllics': 'economics',
 'wllich': 'which',
 'lle': 'he',
 'olle': 'one',
 'llis': 'his',
 'standpoillt': 'standpoint',
 'lleither': 'neither',
 'llor': 'nor',
 'halld': 'hand',
 'alld': 'and',
 'stalldards': 'standards',
 'tllat': 'that',
 'interactioll': 'interaction',
 'whicll': 'which',
 'whell': 'when',
 'natioll': 'nation',
 'illfluence': 'influence',
 'sallle': 'same',
 'higllest': 'highest',
 'scielltific': 'scientific',
 'wllo': 'who',
 'tllose': 'those',
 'llim': 'him',
 'higll': 'high',
 'tllree': 'three',
 'llo': 'no',
 'lllake': 'wake',
 'beillg': 'being',
 'owll': 'own',
 'llave': 'have',
 'mell': 'men',
 'educatioll': 'education',
 'llot': 'not',
 'ollly': 'only'